# Points of Dispense - Flooding in Eastern Allegheny County
#### Minimize total distance and vary number of PODs
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [9]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Optimization

Read in data from files

In [10]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')
bg_included = pd.read_csv('data/bg_pop_included.csv')
pod_data = pd.read_csv('data/POD_ids.csv')

In [11]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [12]:
dist = dist_miles.values
N = population['population'].values
prop = population['flood'].values

Create indices for block groups and PODs

In [13]:
blocks = range(len(N))
pods = range(len(dist[0]))

Vary number of PODs to open

In [14]:
pods_open = range(1,48)
pods_open_results = {}

In [15]:
for i in pods_open: 
    ### Initialize model for POD locations
    m = gp.Model('POD_locations')
    
    ### Add decision variables
    x = m.addVars(pods, vtype=GRB.BINARY, name='x')
    y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='x')
    
    ### Set objective to minimize total distance across the population
    obj = gp.quicksum(dist[j,i] * x[i] * y[j, i] * N[j] * prop[j] 
                  for j in blocks for i in pods)
    m.setObjective(obj, GRB.MINIMIZE)
    
    ### Add constraints
    #  y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened
    m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')
    
    # each block group must be assigned one shelter
    m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')
    
    # number of PODs opened must be less than num_pods
    m.addConstr((gp.quicksum(x[i] for i in pods) <= i), name='pods_opened')
    
    ### Optimize
    print('Solving for:', i, 'PODs')
    m.optimize()
    
    ### Analysis
    block_pod_list = [[j,i] for j in blocks for i in pods if (y[j, i].x==1) & (prop[j]==1)]
    pods_open_results[i] = block_pod_list

Solving for: 1 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 52801 rows, 51747 columns and 155147 nonzeros
Model fingerprint: 0xf783d268
Model has 1880 quadratic objective terms
Variable types: 0 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+02, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 52801 rows and 51747 columns
Presolve time: 1.97s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 2.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 308644 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.086441956233e+05, best bound 3.086441956233e+05, gap 0.0000%
Solving for: 2 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 52801 rows, 51747 columns and 155147 nonzeros
Model fingerprint: 0x7edd06a3

In [17]:
pods_flood_total = open('pods_flood_total','wb') 
pickle.dump(pods_open_results, pods_flood_total)                      
pods_flood_total.close()